In [11]:
import sys
import os
import zipfile
from copy import deepcopy

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'Assignment 1.pdf',
 'q1.py',
 'results',
 'results.zip',
 'task1.csv',
 'task1.ipynb',
 'task2.csv']

In [3]:
# Extract results zip file in scalable way
folder = os.listdir()
try:
    if "results.zip" not in folder:
        print("Results zip file not exists")
        sys.exit()
    elif "results" not in folder:
        with zipfile.ZipFile("results.zip","r") as res_zip:
            res_zip.extractall("results")
            print("Extracting to results folder...")
        print("Extract done")
    else:
        pass
except IOError:
    print("IOError occured, exiting...")
    sys.exit()
    
#results_folder = os.path.join(os.path.dirname(__file__), "results", "results")
results_folder = os.path.join(os.getcwd(), "results", "results")
print(results_folder)
print(len(os.listdir(results_folder)))

C:\Users\wangzhenyu\Desktop\hd\results\results
78


In [36]:
header = ['Condition', 'Name', 'Gender', 'Age', 'Proportion of hits', 'Proportion of nearmisses',
          ' Proportion of fullmisses', 'meanHapp of hit', 'meanWill of hit', 'meanHapp of nearmisses',
          'meanWill of nearmisses', 'meanHapp of fullmisses', 'meanWill of fullmisses', 'maxHapp', 'trialnumber',
          'minHapp', 'trialnumber\n']        
with open("task1.csv", 'w') as dest_file:
    dest_file.write(','.join(header))
    
header = ['Date', 'Number of participants', 'Skill', 'Luck', 'Mixed', 'avg_trials\n']
with open("task2.csv", "w") as dest_file:
    dest_file.write(','.join(header))
    
    
# Global dics
IPs = {}
num_part_date = {}
record_date = {}
record_condition = {}
trial_mappings = {}

# Global vars
file_number = 0

for file in os.listdir(results_folder):
    file_number += 1
    with open(os.path.join(results_folder, file), "r") as f, open("task1.csv", 'a') as t1:
        line_number = 0
        # iterate through one participant file
        trials = 0  # items for Proportion of Hits, Near misses, and Full Misses over the total number of trials
        hits = 0
        near_misses = 0
        full_misses = 0

        happiness_hit = 0  # items for mean happiness and mean willingness
        happiness_full = 0
        happiness_near = 0
        willingness_hit = 0
        willingness_full = 0
        willingness_near = 0
        
        mList = []
        bList = []
        notSameIP = True
        for line in f.readlines():
            line_number += 1
            if line_number == 1:
                line = line.strip('\n')
                #print(line)
                condition = line.strip()
                record_condition[file_number] = condition
            elif line_number == 4:
                infor = line.split(',')
                # print(line)
                Name = infor[0]
                #print(Name)
                # parsing age and gender
                Age = infor[1]
                #print(Age)
                Gender = infor[2]
                #print("Gender", Gender)
                if 'female' in Gender.lower():
                    Gender = "2"
                else:
                    Gender = "1"
            elif line_number == 2:
                _, raw_date = line.strip().split(':')
                raw_date = raw_date.strip()
                date = '-'.join([raw_date[0:2], raw_date[2:4], raw_date[4:8]])
                #print(record_date)
                if date not in record_date:
                    record_date[date] = [file_number]
                else:
                    record_date[date].append(file_number)
                if date not in num_part_date:
                    num_part_date[date] = 1
                else:
                    num_part_date[date] += 1

            # parsing Proportion of Hits, Near misses, and Full Misses over the total number of trials
            elif line_number != 5:
                line = line.strip('\n')
                values = line.split(',')
                #print(values)
                if values[0] != 'trial':
                    continue
                if 'trial' in values:
                    trials = trials + 1
                if 'hit' in values:
                    hits = hits + 1
                if 'fullMiss' in values:
                    full_misses = full_misses + 1
                if 'nearMiss' in values:
                    near_misses = near_misses + 1
                # parsing Mean happiness and mean willingness
                if values[4] == "hit":
                    happiness_hit = happiness_hit + int(values[7])
                    willingness_hit = willingness_hit + int(values[8])
                if values[4] == "fullMiss":
                    happiness_full = happiness_full + int(values[7])
                    willingness_full = willingness_full + int(values[8])
                if values[4] == "nearMiss":
                    happiness_near = happiness_near + int(values[7])
                    willingness_near = willingness_near + int(values[8])
                # parsing the maximum and the minimum reported happiness levels and the trial in which they occurred
                mList.append(int(values[7]))
            elif line_number == 3:
                _, IP = line.strip('\n').split(':')
                if IP.strip() not in IPs:
                    IPs[IP.strip()] = 1
                else:
                    notSameIP = False
                    break
            else:
                continue
                
        if notSameIP:
            # caculation of proportion
            trial_mappings[file_number] = trials
            hit_proportion = str(hits / trials)
            #print(hit_proportion)
            full_proportion = str(full_misses / trials)
            #print(full_proportion)
            near_proportion = str(near_misses / trials)
            #print(near_proportion)
            # caculation of mean
            meanHapp_hit = str(happiness_hit / hits)
            #print(meanHapp_hit)
            meanWill_hit = str(willingness_hit / hits)
            #print(meanWill_hit)
            meanHapp_full = str(happiness_full / full_misses)
            #print(meanHapp_full)
            meanWill_full = str(willingness_full / full_misses)
            #print(meanWill_full)
            meanHapp_near = str(happiness_near / near_misses)
            #print(meanHapp_near)
            meanWill_near = str(willingness_near / near_misses)
            #print(meanWill_near)
            # finding max and min happiness
            #print(mList)
            maxHappiness = str(max(mList))
            #print(maxHappiness)
            trialmax_number = str(mList.index(int(maxHappiness)))
            #print(trialmax_number)
            minHappiness = str(min(mList))
            #print(minHappiness)
            trialmin_number = str(mList.index(int(minHappiness)))
            #print(trialmin_number)

            aList = [condition, Name, Gender, Age, hit_proportion, near_proportion, full_proportion, meanHapp_hit, meanWill_hit, meanHapp_near, meanWill_near, meanHapp_full, meanWill_full, maxHappiness, trialmax_number, minHappiness, trialmin_number]
            #print(aList)
            #print(len(aList))
            bList = ','.join(aList)
            #print("blist\n", bList)
            t1.write(bList + '\n')
            
print(num_part_date)
avg_trials = deepcopy(record_date)
for seq in record_date.values():
    for i in range(len(seq)):
        seq[i] = record_condition[seq[i]]

for k, v in record_date.items():
    mapping = {"skill": 0, "mixed": 0, "luck": 0}
    for ele in v:
        mapping[ele] += 1
    record_date[k] = mapping
print(record_date)
#print(avg_trials)
#print(trial_mappings)

for k, v in avg_trials.items():
    avg_trials[k] = round(sum(map(lambda x: trial_mappings[x], v)) / len(v), 2)

print(avg_trials)
print(num_part_date.keys())
print(record_date.keys())
print(avg_trials.keys())
with open("task2.csv", 'a') as t2:
    for date in num_part_date.keys():
        record = [date, str(num_part_date[date]), str(record_date[date]['skill']), \
                  str(record_date[date]['luck']), str(record_date[date]['mixed']), str(avg_trials[date])]
        t2.write(','.join(record))
        t2.write('\n')
        
        #def singlePartInfo(file, dest):
    

{'04-06-2013': 16, '11-06-2013': 8, '10-06-2013': 9, '05-06-2013': 4, '12-06-2013': 19, '13-06-2013': 12, '06-06-2013': 5, '17-06-2013': 3, '18-06-2013': 1, '07-06-2013': 1}
{'04-06-2013': {'skill': 6, 'mixed': 5, 'luck': 5}, '11-06-2013': {'skill': 3, 'mixed': 2, 'luck': 3}, '10-06-2013': {'skill': 3, 'mixed': 4, 'luck': 2}, '05-06-2013': {'skill': 2, 'mixed': 1, 'luck': 1}, '12-06-2013': {'skill': 6, 'mixed': 6, 'luck': 7}, '13-06-2013': {'skill': 5, 'mixed': 4, 'luck': 3}, '06-06-2013': {'skill': 1, 'mixed': 2, 'luck': 2}, '17-06-2013': {'skill': 1, 'mixed': 1, 'luck': 1}, '18-06-2013': {'skill': 1, 'mixed': 0, 'luck': 0}, '07-06-2013': {'skill': 0, 'mixed': 1, 'luck': 0}}
{'04-06-2013': 26.75, '11-06-2013': 26.88, '10-06-2013': 26.67, '05-06-2013': 26.5, '12-06-2013': 26.68, '13-06-2013': 26.33, '06-06-2013': 27.4, '17-06-2013': 27.67, '18-06-2013': 29.0, '07-06-2013': 26.0}
dict_keys(['04-06-2013', '11-06-2013', '10-06-2013', '05-06-2013', '12-06-2013', '13-06-2013', '06-06-2013',

In [5]:
print(num_part_date)
print(sum(num_part_date.values()))

{'04-06-2013': 16, '11-06-2013': 8, '10-06-2013': 9, '05-06-2013': 4, '12-06-2013': 19, '13-06-2013': 12, '06-06-2013': 5, '17-06-2013': 3, '18-06-2013': 1, '07-06-2013': 1}
78


In [6]:
import pandas as pd
os.path.join(os.getcwd(), "results", "results", "results_10.csv")

'C:\\Users\\wangzhenyu\\Desktop\\hd\\results\\results\\results_10.csv'

In [7]:
file = pd.read_csv("results/results/results_10.csv", encoding="utf-8", sep='\t')

In [8]:
file.head(20)

,skill
0,Date: 04062013144323
1,IP address: 69.216.67.223
2,"Manoj,25,male"
3,",Ships,Aimed For,Landed At,Result,Latency,Move..."
4,"trial,6:1|6:2|1:3|1:4|5:8|6:8|,1:4,2:5,nearMis..."
5,"trial,5:4|6:4|4:8|5:8|,4:8,2:9,fullMiss,10.597..."
6,"trial,4:3|4:4|8:4|8:5|4:8|4:9|,4:9,4:8,hit,4.1..."
7,"trial,6:7|7:7|7:3|8:3|1:8|2:8|,2:8,3:9,nearMis..."
8,"trial,5:2|5:3|3:8|3:9|7:7|7:8|,3:8,3:8,hit,6.0..."
9,"trial,4:7|5:7|9:3|9:4|3:2|4:2|,4:7,2:6,fullMis..."
